In [1]:
import sympy as sp
import numpy as np

# Definir variables
x, y, z = sp.symbols('x y z')

# Ejemplo de datos de entrada y valores objetivo
data = [
    {x: 1, y: 2, z: 3, 'target': 10},
    {x: 4, y: 5, z: 6, 'target': 35},
]

# Clase Nodo para representar el árbol
class Nodo:
    def __init__(self, valor, izq=None, der=None):
        self.valor = valor
        self.izq = izq
        self.der = der

# Función para imprimir el árbol
def imprimir_arbol(nodo, nivel=0):
    if nodo is not None:
        imprimir_arbol(nodo.der, nivel + 1)
        print(' ' * 4 * nivel + '->', nodo.valor)
        imprimir_arbol(nodo.izq, nivel + 1)

# Función para evaluar el árbol
def evaluar_arbol(nodo, variables):
    if nodo.izq is None and nodo.der is None:
        if isinstance(nodo.valor, sp.Symbol) or isinstance(nodo.valor, str):
            return variables[nodo.valor]  # Valor de la variable
        else:
            return nodo.valor  # Número
    
    izq_val = evaluar_arbol(nodo.izq, variables)
    der_val = evaluar_arbol(nodo.der, variables)
    
    if nodo.valor == '+':
        return izq_val + der_val
    elif nodo.valor == '-':
        return izq_val - der_val
    elif nodo.valor == '*':
        return izq_val * der_val
    elif nodo.valor == '/':
        return izq_val / der_val
    else:
        raise ValueError(f"Operador no soportado: {nodo.valor}")

# Función de error
def calcular_error(arbol, data):
    total_error = 0
    for entrada in data:
        # Evaluar el árbol con los valores de entrada
        resultado = evaluar_arbol(arbol, entrada)
        objetivo = entrada['target']
        total_error += (resultado - objetivo) ** 2  # Error cuadrático
    return total_error  # Evaluar el total error como un número

# Crear un árbol inicial
arbol_inicial = Nodo("*",
                     Nodo("+", Nodo(x), Nodo(3)),
                     Nodo("-", Nodo(y), Nodo(5)))

# Calcular el error inicial
error_inicial = calcular_error(arbol_inicial, data)
print("Error inicial:", error_inicial)

# Imprimir el árbol
print("\nÁrbol original:")
imprimir_arbol(arbol_inicial)

# Función para reemplazar una variable en el árbol y minimizar el error
def reemplazar_y_minimizar(arbol, variable, nuevas_expresiones, data):
    mejor_arbol = arbol
    mejor_error = calcular_error(arbol, data)  # Evalúa el error inicial como un número

    for nueva_expr in nuevas_expresiones:
        # Crear un nuevo árbol reemplazando el nodo hoja `variable`
        nuevo_arbol = Nodo("*",
                           Nodo("+", nueva_expr, Nodo(3)),  # Usar la nueva expresión como un nodo
                           Nodo("-", Nodo(y), Nodo(5)))

        # Calcular el nuevo error
        error_nuevo = calcular_error(nuevo_arbol, data)  # Evalúa el nuevo error como un número

        # Si el nuevo error es mejor, actualizar mejor_arbol
        if error_nuevo < mejor_error:
            mejor_error = error_nuevo
            mejor_arbol = nuevo_arbol

    return mejor_arbol, mejor_error

# Definir nuevas expresiones para probar
nuevas_expresiones = [
    Nodo('+', Nodo(x), Nodo(1)),  # x + 1
    Nodo('-', Nodo(x), Nodo(2)),  # x - 2
    Nodo('*', Nodo(x), Nodo(2)),  # x * 2
    Nodo('/', Nodo(x), Nodo(2)),  # x / 2
]

# Reemplazar x y minimizar el error
arbol_optimo, error_optimo = reemplazar_y_minimizar(arbol_inicial, x, nuevas_expresiones, data)

# Imprimir resultados
print("\nÁrbol óptimo:")
imprimir_arbol(arbol_optimo)
print("Error óptimo:", error_optimo)


Error inicial: 1709

Árbol original:
        -> 5
    -> -
        -> y
-> *
        -> 3
    -> +
        -> x

Árbol óptimo:
        -> 5
    -> -
        -> y
-> *
        -> 3
    -> +
            -> 2
        -> -
            -> x
Error óptimo: 1481


In [2]:
import random

def generar_nuevas_expresiones_aleatorias(variables, operaciones, num_expresiones=5, rango_numeros=(-10, 10)):
    nuevas_expresiones = []
    
    for _ in range(num_expresiones):
        # Elegir una operación aleatoria
        op = random.choice(operaciones)

        # Elegir un número aleatorio dentro del rango especificado
        num_aleatorio = random.uniform(rango_numeros[0], rango_numeros[1])

        # Elegir una variable aleatoria
        variable = random.choice(variables)

        # Crear la nueva expresión
        if op == '+':
            nuevas_expresiones.append(Nodo('+', Nodo(variable), Nodo(num_aleatorio)))
        elif op == '-':
            nuevas_expresiones.append(Nodo('-', Nodo(variable), Nodo(num_aleatorio)))
        elif op == '*':
            nuevas_expresiones.append(Nodo('*', Nodo(variable), Nodo(num_aleatorio)))
        elif op == '/':
            # Evitar división por cero
            if num_aleatorio != 0:
                nuevas_expresiones.append(Nodo('/', Nodo(variable), Nodo(num_aleatorio)))


    return nuevas_expresiones

# Definir operaciones que quieras utilizar
operaciones = ['+', '-', '*', '/']
variables= [x, y, z]  # Puedes agregar más variables si lo deseas

# Generar nuevas expresiones aleatorias
nuevas_expresiones = generar_nuevas_expresiones_aleatorias(variables, operaciones)

# Reemplazar x y minimizar el error usando las nuevas expresiones generadas
arbol_optimo, error_optimo = reemplazar_y_minimizar(arbol_inicial, x, nuevas_expresiones, data)

# Imprimir resultados

# Calcular el error inicial
error_inicial = calcular_error(arbol_inicial, data)
print("Error inicial:", error_inicial)

# Imprimir el árbol
print("\nÁrbol original:")
imprimir_arbol(arbol_inicial)


print("\nÁrbol óptimo:")
imprimir_arbol(arbol_optimo)
print("Error óptimo:", error_optimo)


Error inicial: 1709

Árbol original:
        -> 5
    -> -
        -> y
-> *
        -> 3
    -> +
        -> x

Árbol óptimo:
        -> 5
    -> -
        -> y
-> *
        -> 3
    -> +
            -> -5.626524905714909
        -> *
            -> x
Error óptimo: 1229.496203380172


In [3]:
import random
import sympy as sp

# Definir variables
x, y, z = sp.symbols('x y z')

# Clase Nodo para representar el árbol
class Nodo:
    def __init__(self, valor, izq=None, der=None):
        self.valor = valor
        self.izq = izq
        self.der = der

# Función para imprimir el árbol
def imprimir_arbol(nodo, nivel=0):
    if nodo is not None:
        imprimir_arbol(nodo.der, nivel + 1)
        print(' ' * 4 * nivel + '->', nodo.valor)
        imprimir_arbol(nodo.izq, nivel + 1)

# Función para evaluar el árbol
def evaluar_arbol(nodo, variables):
    if nodo.izq is None and nodo.der is None:
        return variables[nodo.valor] if isinstance(nodo.valor, sp.Symbol) else nodo.valor
    
    izq_val = evaluar_arbol(nodo.izq, variables)
    der_val = evaluar_arbol(nodo.der, variables)
    
    if nodo.valor == '+':
        return izq_val + der_val
    elif nodo.valor == '-':
        return izq_val - der_val
    elif nodo.valor == '*':
        return izq_val * der_val
    elif nodo.valor == '/':
        return izq_val / der_val
    else:
        raise ValueError(f"Operador no soportado: {nodo.valor}")

# Función de error
def calcular_error(arbol, data):
    total_error = 0
    for entrada in data:
        resultado = evaluar_arbol(arbol, entrada)
        objetivo = entrada['target']
        total_error += (resultado - objetivo) ** 2
    return total_error

# Generar nuevas expresiones aleatorias
def generar_nuevas_expresiones_aleatorias(variables, operaciones, num_expresiones=5, rango_numeros=(-10, 10)):
    nuevas_expresiones = []
    
    for _ in range(num_expresiones):
        op = random.choice(operaciones)
        num_aleatorio = random.uniform(rango_numeros[0], rango_numeros[1])
        variable = random.choice(variables)

        if op == '+':
            nuevas_expresiones.append(Nodo('+', Nodo(variable), Nodo(num_aleatorio)))
        elif op == '-':
            nuevas_expresiones.append(Nodo('-', Nodo(variable), Nodo(num_aleatorio)))
        elif op == '*':
            nuevas_expresiones.append(Nodo('*', Nodo(variable), Nodo(num_aleatorio)))
        elif op == '/':
            if num_aleatorio != 0:
                nuevas_expresiones.append(Nodo('/', Nodo(variable), Nodo(num_aleatorio)))

    return nuevas_expresiones

# Definir operaciones que quieras utilizar
operaciones = ['+', '-', '*', '/']
variables = [x, y, z]  # Puedes agregar más variables si lo deseas

# Crear un árbol inicial
arbol_inicial = Nodo("*",
                     Nodo("+", Nodo(x), Nodo(3)),
                     Nodo("-", Nodo(y), Nodo(5)))

# Calcular el error inicial
error_inicial = calcular_error(arbol_inicial, data)
print("Error inicial:", error_inicial)

# Imprimir el árbol
print("\nÁrbol original:")
imprimir_arbol(arbol_inicial)

# Inicializar variables para la optimización
arbol_optimo = arbol_inicial  # Comenzar con el árbol inicial
error_optimo = error_inicial  # Comenzar con el error inicial
mejor_error = error_optimo
iteracion = 0
max_iteraciones = 10000
max_convergencias = 100
cont_convergencia = 0

# Proceso de optimización
while (iteracion < max_iteraciones) and (cont_convergencia < max_convergencias):
    # Generar nuevas expresiones aleatorias
    nuevas_expresiones = generar_nuevas_expresiones_aleatorias(variables, operaciones)
    arbol_optimo, error_optimo = reemplazar_y_minimizar(arbol_optimo, variables, nuevas_expresiones, data)
    
    if (mejor_error - error_optimo) < 1e-16:
        cont_convergencia += 1
    else:
        cont_convergencia = 0
        mejor_error = error_optimo
    
    iteracion += 1
    print(iteracion, error_optimo, cont_convergencia)

# Imprimir resultados
print("\nÁrbol óptimo:")
imprimir_arbol(arbol_optimo)
print("Error óptimo:", error_optimo)


Error inicial: 1709

Árbol original:
        -> 5
    -> -
        -> y
-> *
        -> 3
    -> +
        -> x
1 1227.7479280989114 0
2 1227.7479280989114 1
3 1227.7479280989114 2
4 1227.7479280989114 3
5 1227.7479280989114 4
6 1227.7479280989114 5
7 1227.7479280989114 6
8 1227.7479280989114 7
9 1225.5199095731327 0
10 1225.5199095731327 1
11 1225.5199095731327 2
12 1225.5199095731327 3
13 1225.5199095731327 4
14 1225.5199095731327 5
15 1225.5199095731327 6
16 1225.5199095731327 7
17 1225.5199095731327 8
18 1225.0387363172647 0
19 1225.0387363172647 1
20 1225.0387363172647 2
21 1225.0387363172647 3
22 1225.0387363172647 4
23 1225.0387363172647 5
24 1225.0387363172647 6
25 1225.0387363172647 7
26 1225.0387363172647 8
27 1225.0387363172647 9
28 1225.0013002056594 0
29 1225.0013002056594 1
30 1225.0013002056594 2
31 1225.0013002056594 3
32 1225.0013002056594 4
33 1225.0013002056594 5
34 1225.0013002056594 6
35 1225.0013002056594 7
36 1225.0013002056594 8
37 1225.0013002056594 9
38 1225.0

In [51]:
import tensorflow as tf

# Clase Nodo para representar el árbol
class Nodo:
    def __init__(self, valor, izq=None, der=None):
        self.valor = valor  # Puede ser un operador o un valor numérico
        self.izq = izq      # Hijo izquierdo
        self.der = der      # Hijo derecho
 
# Función para evaluar el árbol
def evaluar_arbol(nodo, variables):
    if nodo.izq is None and nodo.der is None:  # Nodo hoja
        return variables[nodo.valor] if isinstance(nodo.valor, str) else nodo.valor
    
    # Evaluar hijos
    izq_val = evaluar_arbol(nodo.izq, variables)
    der_val = evaluar_arbol(nodo.der, variables)
    
    # Operaciones
    if nodo.valor == '+':
        return izq_val + der_val
    elif nodo.valor == '-':
        return izq_val - der_val
    elif nodo.valor == '*':
        return izq_val * der_val
    elif nodo.valor == '/':
        return izq_val / der_val
    else:
        raise ValueError(f"Operador no soportado: {nodo.valor}")

# Función para calcular la pérdida del subárbol
def calcular_perdida_subarbol(subarbol, data):
    total_perdida = 0.0
    for entrada in data:
        x, y, z, objetivo = entrada['x'], entrada['y'], entrada['z'], entrada['target']
        prediccion = evaluar_arbol(subarbol, {'x': x, 'y': y, 'z': z})
        total_perdida += (prediccion - objetivo) ** 2
    return total_perdida / len(data)

# Datos de ejemplo
data = [{'x': 1, 'y': 2, 'z': 3, 'target': 10},
        {'x': 4, 'y': 5, 'z': 6, 'target': 35},
        {'x': 7, 'y': 8, 'z': 9, 'target': 50}]

# Crear el árbol inicial
# Utilizar tf.Variable para las variables que se optimizarán
valor_izq = tf.Variable(1.0, dtype=tf.float32)  # Valor que será optimizado
valor_der = tf.Variable(3.0, dtype=tf.float32)  # Otro valor a optimizar
nodo_izq = Nodo('+', Nodo(x), Nodo(y))  # Usar 'x' y 'y' como variables
nodo_der = Nodo('*', Nodo(valor_izq), Nodo(valor_der))  # Usar variables optimizables
arbol_inicial = Nodo('-', nodo_izq, nodo_der)

# Extraer el subárbol desde un nodo específico
nodo_subarbol = arbol_inicial.der  # Suponiendo que optimizamos el subárbol derecho

# Optimización
optimizador = tf.keras.optimizers.Adam(learning_rate=0.01)

# Entrenamiento
for iteracion in range(1000):
    with tf.GradientTape() as tape:
        perdida = calcular_perdida_subarbol(nodo_subarbol, data)

    # Obtener los gradientes de las variables en el subárbol
    gradientes = tape.gradient(perdida, [valor_izq, valor_der])
    optimizador.apply_gradients(zip(gradientes, [valor_izq, valor_der]))

    if iteracion % 100 == 0:
        print(f"Iteración {iteracion}, Pérdida: {perdida.numpy()}")

# Mostrar el resultado final
print(f"Pérdida final: {perdida.numpy()}")
print(f"Valor optimizado izquierdo: {valor_izq.numpy()}")
print(f"Valor optimizado derecho: {valor_der.numpy()}")

def imprimir_arbol(nodo, nivel=0):
    if nodo is not None:
        # Imprimir el subárbol derecho primero
        imprimir_arbol(nodo.der, nivel + 1)
        
        # Imprimir el valor del nodo actual
        indentacion = '   ' * nivel  # Indentación para el nivel actual
        valor = nodo.valor.numpy() if isinstance(nodo.valor, tf.Variable) else nodo.valor  # Obtener el valor numérico
        print(f"{indentacion}-> {valor}")  # Mostrar el valor con el prefijo
        
        # Imprimir el subárbol izquierdo
        imprimir_arbol(nodo.izq, nivel + 1)

# Ejemplo de uso
print("Estructura del árbol:")
imprimir_arbol(arbol_inicial)
print("Estructura del árbol inicial:")
imprimir_arbol(aux)



Iteración 0, Pérdida: 1094.0
Iteración 100, Pérdida: 820.126708984375
Iteración 200, Pérdida: 524.1907348632812
Iteración 300, Pérdida: 338.40936279296875
Iteración 400, Pérdida: 280.622802734375
Iteración 500, Pérdida: 272.7309875488281
Iteración 600, Pérdida: 272.2385559082031
Iteración 700, Pérdida: 272.2225341796875
Iteración 800, Pérdida: 272.22222900390625
Iteración 900, Pérdida: 272.2221984863281
Pérdida final: 272.22222900390625
Valor optimizado izquierdo: 4.514819145202637
Valor optimizado derecho: 7.01392126083374
Estructura del árbol:
      -> 7.01392126083374
   -> *
      -> 4.514819145202637
-> -
      -> y
   -> +
      -> x
Estructura del árbol inicial:
      -> 7.01392126083374
   -> *
      -> 4.514819145202637
-> -
      -> y
   -> +
      -> x


In [49]:
arbol_inicial.der.der.valor.numpy()*arbol_inicial.der.der.valor.numpy()

49.19509

In [ ]:
arbol_inicial